In [3]:
import numpy as np
import pandas as pd
import os, sys, gc, warnings, random, datetime

from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold, GroupKFold
from sklearn.preprocessing import LabelEncoder

from catboost import CatBoostClassifier

warnings.filterwarnings('ignore')

In [9]:
train=pd.read_csv('train_feat1.csv',index_col=0)
test=pd.read_csv('test_feat1.csv',index_col=0)
Y_train=train.rent
X_train=train.drop(['id','rent'],axis=1)
X_test=test.drop(['id','rent'],axis=1)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31470 entries, 0 to 31469
Columns: 117 entries, id to period_num
dtypes: float64(2), int64(115)
memory usage: 29.6 MB


In [5]:
cat_params = {
                'n_estimators':5000,
                'learning_rate': 0.07,
                'eval_metric':'AUC',
                'loss_function':'Logloss',
                'random_seed':0,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 8,
                #'colsample_bylevel':0.7,
                } 

NameError: name 'train' is not defined

In [6]:
splits = 5
folds = KFold(n_splits=splits)

aucs = list()
feature_importances = pd.DataFrame()
feature_importances['feature'] = X_train.columns

training_start_time = time()

y_preds = np.zeros(X_test.shape[0])

for fold_n, (trn_idx, val_idx) in enumerate(folds.split(X_train,Y_train)):
    cv_fold_start_time = time()
    print ('** Training fold {}'.format(fold_n + 1))
    X_trn, X_val = X_train.iloc[trn_idx], X_train.iloc[val_idx]
    y_trn, y_val = Y_train[trn_idx], Y_train[val_idx]
    eval_set  = [(X_trn,y_trn), (X_val, y_val)]
    
    reg = CatBoostClassifier(**cat_params)        
    reg.fit(
        X_trn,y_trn,
        eval_set=(X_val, y_val),
        cat_features=categorical_features,
        use_best_model=True,
        verbose=True)
    
    del X_trn, y_trn
    
    val=reg.predict(X_val)
    feature_importances['fold_{}'.format(fold_n + 1)] = reg.feature_importances_
    
    y_preds=reg.predict(X_test)/splits
    del reg, X_val
    print('RMSE accuracy: {}'.format(rmse(y_val, val)))
    del val,y_val

    gc.collect()
    
    cv_fold_end_time = time()
    print ('fold completed in {}s'.format(cv_fold_end_time - cv_fold_start_time))

NameError: name 'train_df' is not defined